![Landsat8](./images/nasa_landsat8.jpg "Landsat8")

# Data Ingestion - Intake

---

## Overview

In the last notebook, you learned how to efficiently load data from the Microsoft Planetary Computer platform. If that approach works for you, please proceed to a workflow example. However, there are many other established approaches and platforms (e.g. [NASA EarthData](https://search.earthdata.nasa.gov/search), [USGS EarthExplorer](https://earthexplorer.usgs.gov/), [Google](https://developers.google.com/earth-engine/datasets/catalog/landsat), and [AWS](https://registry.opendata.aws/usgs-landsat/)),  for loading Landsat data that may not preprocess the data in the exact same way or use a similar API. For this reason, in this notebook we will demonstrate common alternative approaches and techniques for general data access, centered around [Intake](https://intake.readthedocs.io).

First, to demonstrate simple cases, we will begin by reading in a local files with [Pandas](https://pandas.pydata.org/docs/) and [xarray](https://docs.xarray.dev/en/stable/). Then we will demonstrate the combined use of [Intake](https://intake.readthedocs.io) and [Dask](https://docs.dask.org/en/stable/) libraries to efficiently fetch data from a remote server.

<div class="admonition alert alert-info">
    <p class="admonition-title" style="font-weight:bold">Info</p>
        A great way to contribute to this cookbook is to create a notebook that focuses on data access from a specific provider.
</div>

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to Landsat](./0.0_Intro_Landsat.ipynb) | Necessary | Background |
| [Data Ingestion - Planetary Computer](1.0_Data_Ingestion-Planetary_Computer.ipynb) | Helpful | |
| [Pandas Cookbook](https://foundations.projectpythia.org/core/pandas.html) | Helpful |  |
| [xarray Cookbook](https://foundations.projectpythia.org/core/xarray.html) | Necessary |  |
| [Intake Quickstart](https://intake.readthedocs.io/en/latest/index.html) | Helpful |  |
|[Intake Cookbook](https://projectpythia.org/intake-cookbook/README.html)| Necessary | |

- **Time to learn**: 20 minutes

---

## Imports

In [ ]:
import pandas as pd
import xarray as xr
import intake
import hvplot.intake
import hvplot.xarray

import warnings
warnings.simplefilter('ignore', FutureWarning) # Ignore warning about the format of epsg codes

## Loading local metadata with Pandas

Often, it's common to have local files that could contain metadata.  Therefore, we'll start with the simple case of loading small local tabular data, such as `.csv` files into Pandas Dataframes. 

Let's load information about Landsat 5 bands:

In [ ]:
landsat5_bands = pd.read_csv('data/landsat5_bands.csv').set_index('Band')
landsat5_bands

...and the relevant subset of Landsat 8 bands: 

In [ ]:
landsat8_bands = pd.read_csv('data/landsat8_bands.csv').set_index('Band')
landsat8_bands

Interestingly, we can see that the range and index of the bands differ between the satellites. For example, 'Red' is Band-4 within Landsat 8, but Band-3 within Landsat 5. This is important to keep in mind whenever comparing data across Landsat missions.

## Loading small and local data with xarray

Although initiatives are underway to enhance cloud-based data workflows, at some point, it will likely be the case that you will have some local gridded data that you want to load and preview without much fuss. For this simple case, it makes sense to use an appropriately simple API. Using xarray and hvplot, we can quickly complete this task of previewing a local [netCDF](https://foundations.projectpythia.org/core/data-formats/netcdf-cf.html) file.

In [ ]:
landsat_5_crop = xr.open_dataarray('data/landsat5_crop.nc')
landsat_5_crop

In [ ]:
landsat_5_crop_image = landsat_5_crop.hvplot.image(geo=True, tiles='ESRI', alpha=.7)
landsat_5_crop_image

This image is just a cropped portion of a Landsat tile, solely for demonstration of loading with xarray. Also, as we learned in the last notebook, as long as the data has a valid `crs` attribute, passing `geo=True` to [hvPlot](https://hvplot.holoviz.org/user_guide/Geographic_Data.html) will plot the data in a geographic coordinate system, allowing us to use the `tiles` option to overlay the plot on top of map tiles.

## Loading data of any size and location using Intake

Even if your current data files are local, earth science data is often complex, remote, and large, so it's a good idea to become familiar with an approach that can handle data of nearly any complexity, location, and size. For this reason, we will demonstrate a couple different approaches of the [Intake](https://intake.readthedocs.io) library. 

First, we will start with a simpler case of reading the same local tabular file as above with Pandas. Then we will progressively cover more complex cases, such as loading multiple files and loading gridded data from a remote location with an Intake catalog.

In [ ]:
ds = intake.open_csv('./data/landsat5_bands.csv')

Importantly, the data is not loaded yet - `open_csv` only accesses the metadata:

In [ ]:
ds

We can also peak at what format the data will be in when it gets loaded using `.container`:

In [ ]:
ds.container

Now, to get better insight into the data without loading it all in just yet, we can inspect the data using `.to_dask()`. You could also use `.read_chunked()` to return an iterator over the chunks (or partitions) of data. Refer to the ['Reading Data'](https://intake.readthedocs.io/en/latest/quickstart.html?highlight=read_chunked#reading-data) Intake documentation for more information about these options.

In [ ]:
dd = ds.to_dask()
dd.head()

In [ ]:
dd.info()

Notice that now running `.info()` gives us a lot less information than when we loaded the data earlier using `pd.read_csv`, this is because the `Dask` representation of the data only loads what it needs, when it is needed for a computation. However, you can always explicitly load all the data into memory using `.read()` (but be careful to only run this on a dataset that will fit on your computer's memory). Our sample dataset is very small, so let's try it:

In [ ]:
df = ds.read()
df.info()

### Loading multiple local files

Intake also lets the user load and concatenate data across multiple files in one command

In [ ]:
ds = intake.open_csv(['./data/landsat5_bands.csv', './data/landsat8_bands.csv'])

If the data file names allow, this can be more simply expressed using `*`:

In [ ]:
ds = intake.open_csv('./data/landsat*_bands.csv')

In [ ]:
df = ds.read()
df.info()

But sometimes there is data encoded in a file name or path that causes concatenated data to lose some important context. In this example, we lose the information about which version of landsat the training was done on (5 vs 8). To keep track of that information, we can use a [python format string](https://docs.python.org/3/library/string.html#format-string-syntax) `{variable:type}` to specify our path and declare a new field on our data. That field will get populated based on its value in the path.

In [ ]:
ds = intake.open_csv('./data/landsat{Landsat:d}_bands.csv')
df = ds.read().set_index(['Landsat', 'Band'])
df

### Loading remote data with a catalog

Probably the most versatile and reproducible approach to loading data with Intake is to use a simple Catalog file to declare how the data should be loaded. The catalog lays out how the data should be loaded, defines some metadata, and specifies any patterns in the file path that should be included in the data. Here is an example of an entry in a catalog YAML file:

``` yaml
sources:
  landsat_5_small:
    description: Small version of Landsat 5 Surface Reflectance Level-2 Science Product.
    driver: rasterio
    cache:
      - argkey: urlpath
        regex: 'earth-data/landsat'
        type: file
    args:
      urlpath: 's3://earth-data/landsat/small/LT05_L1TP_042033_19881022_20161001_01_T1_sr_band{band:d}.tif'
      chunks:
        band: 1
        x: 50
        y: 50
      concat_dim: band
      storage_options: {'anon': True}
```

The `urlpath` can be a path to a file, list of files, or a path with [glob](https://en.wikipedia.org/wiki/Glob_(programming)) notation. Alternatively, the path can be written as a python style [format_string](https://docs.python.org/3/library/string.html#format-string-syntax). In the case where the `urlpath` is a format string (for example, `{band:d}`), the fields specified in that string will be parsed from the filenames and returned in the data.

Let's look at what our catalog contains using `open_catalog`:

In [ ]:
cat = intake.open_catalog('./data/catalog.yml')
list(cat)

Here, we see a list of datasets specified by the catalog.yml file. Lets's look at a `description` and the `urlpath` for one of them:

In [ ]:
landsat_5 = cat.landsat_5_small

In [ ]:
landsat_5.description

In [ ]:
landsat_5.container

From `.container` we can see that ultimately the data will be an xarray object

In [ ]:
landsat_5.urlpath

From the `urlpath` we can see that this dataset is hosted remotely on Amazon S3

Let's preview this dataset using `.to_dask()`.

In [ ]:
landsat_5 = cat.landsat_5_small
landsat_5.to_dask()

From this information we can see that there are three dimensions with coordinates `x`, `y`, and `band`. This is satellite data where the `x` and `y` are spatial earth coordinates and the `band` refers to the wavelength range of light used to capture the satellite image.

### Visualizing the data

Often, data ingestion involves quickly visualizing your raw data to get a sense that things are proceeding accordingly. You have already seen how we can easily use [hvPlot](https://hvplot.pyviz.org/) for bare xarray data, but hvPlot also provides interactive plotting commands for Intake, Pandas, Dask, and GeoPandas objects. We'll look more closely at hvPlot and its options in later tutorials.

For now, let's just plot out our different band images as columns (using `col='band'`). We can also set `rasterize=True` to use [Datashader](https://datashader.org/) (another HoloViz tool) to render large data into a 2D histogram, where every array cell counts the data points falling into that pixel, as set by the resolution of your screen.

In [ ]:
landsat_5.hvplot.image(col='band', cmap='viridis', xaxis=False, yaxis=False, colorbar=False, rasterize=True)

Hint: Try zooming in on one plot to see that they are all linked!

If we increase the complexity a bit and consider the likely situation where you are constantly loading and visualizing a similar type of data using an Intake catalog, you can instead just declare plotting parameters directly into the catalog!

Here is the relevant part of our `catalog.yml`:

``` yaml
metadata:
  plots:
    band_image:
      kind: 'image'
      x: 'x'
      y: 'y'
      groupby: 'band'
      rasterize: True
      width: 400
      dynamic: False
```

In [ ]:
landsat_5 = cat.landsat_5_small
landsat_5.to_dask()

In [ ]:
landsat_5.hvplot.band_image()

Excellent! Since we didn't specify that the images should to be plotted as columns, hvPlot conveniently gives us a widget to scrub through them.

## Side-note: Drilling down for data access

As part of accessing data, it's important to understand how to drill down to different representations of our data that certain processing steps might require.

### Numpy Array

Earlier, we saw that if the selected data are small enough, we can load return all the data into one in-memory xarray DataArray container. Once in an DataArray object, the data can be easily manipulated and visualized. Under the hood, this xarray object is essentially comprised of NumPy arrays with the addition of coordinate labels (and [other superpowers](https://docs.xarray.dev/en/latest/getting-started-guide/why-xarray.html)). However, sometimes machine learning tools (e.g. [scikit-learn](https://scikit-learn.org/stable/)) only accept NumPy arrays as input. These arrays are accessible to us from the xarray DataArray objects by using the `.values` attribute.

In [ ]:
npa = cat.landsat_5_small.read().values
type(npa)

In [ ]:
npa.shape

In [ ]:
npa[:3, :3, :3]

## Side-note: Organizing Multiple Datasets into a DataTree

In addition to drilling down to NumPy Arrays for data access, we can also zoom out to use an even higher level container that an xarray Dataset. The [Datatree library](https://xarray-datatree.readthedocs.io/en/latest/index.html) provides a tree-like container that can hold xarray Datasets that are organised into arbitrarily nested groups like a Python dictionary. This can come in handy if you are working with several DataSets that are **not alignable** (i.e. they live on different grids), but are still related to each other. For example, you may have Landsat data from different missions or on a different underlying grid/resolution. The main benefit of using a DataTree is that you can map standard xarray operations recursively over all nodes.


<div class="admonition alert alert-warning">
    <p class="admonition-title" style="font-weight:bold">Warning</p>
    DataTree is currently in incubation and may be upstreamed into xarray in the future. Check the status on the <a href="https://github.com/xarray-contrib/datatree">Datatree repository</a> before use
</div>

Let's import datatree and load datasets from different missions that have different underlying grids, resolutions, and offsets, but cover the same lake at different times.

In [ ]:
from datatree import DataTree

In [ ]:
ls5_da = cat.landsat_5_small.read()
ls5_da.head(2)

In [ ]:
ls8_da = cat.landsat_8_small.read()
ls8_da.head(2)

These two sets of Landsat bands cover roughly the same area but were taken in 1988 and 2017. They have different resolutions, different numbers of grid cells per image, and different x and y offsets. We can see the offset by printing the first `x` value for each year and seeing that they are not equivalent.

In [ ]:
print(f'Landsat 5 data first x coord: {ls5_da.x[0].values}')
print(f'Landsat 8 data first x coord: {ls8_da.x[0].values}')

Before we organize the data into a DataTree, let's promote our DataArrays into DataSets. Each node of the DataTree contains exactly one xarray DataSet.

In [ ]:
landsat_8_ds = landsat_8.to_dataset(name='Walker Lake')
landsat_5_ds = landsat_8.to_dataset(name='Walker Lake')

Now let's combine our DataSets into Landsat subgroups by mission. We could arbitrarily nest these groups as our research requires.

In [ ]:
dt = DataTree.from_dict({"Landsat/Mission8": landsat_8_ds, "Landsat/Mission5": landsat_5_ds})
dt

Now we could map operations across our Landsat DataTree nodes, even though they are misaligned.

In [ ]:
dt.mean(dim='x')

This was a very simple demonstration of DataTree. Check the [DataTree documentation](https://xarray-datatree.readthedocs.io/en/latest/quick-overview.html) for a comprehensive overview.

---

## Summary
Before accessing any data, it's a good idea to start by learning about the context and details of the dataset. This will give you the intuition to make informed decisions as you form a processing and analysis pipeline. Once you have identified a dataset to use, tailor the data access approach given the features of the data, such as size and location. If your data is local and small, a reliable approach is to access the data with either Pandas or xarray, depending on whether the data is tabular or gridded. As earth science data is often large, multidimensional, and on a remote server, a powerful and increasingly common approach is to use the combination of Intake, Dask, and xarray. Once you have accessed data, visualize it with hvPlot to ensure that it matches your expectations.

### What's next?
After loading the data, you may need to process it in some way before it can be fed into your analysis workflow. We will cover an example of such processing in the next section.

## Resources and references
- Authored/adapted by Demetris Roumis circa Dec, 2022
- The Landsat 8 banner image is from [NASA](https://www.nasa.gov/mission_pages/landsat/main/LDCM_Spacecraft_intro.html)
